# Day 4

## Tokenizing with code

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='chatgpt-image-latest', created=1765925279, object='model', owned_by='system'), Model(id='gpt-4o-mini-tts-2025-03-20', created=1765610731, object='model', owned_by='system'), Model(id='gpt-4o-mini-tts-2025-12-15', created=1765610837, object='model', owned_by='system'), Model(id='gpt-realtime-mini-2025-12-15', created=1765612007, object='model', owned_by='system'), Model(id='gpt-audio-mini-2025-12-15', created=1765760008, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turb

In [5]:
for m in client.models.list().data:
    print(m.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
chatgpt-image-latest
gpt-4o-mini-tts-2025-03-20
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
computer-use-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
computer-use-preview-2025-03-11
gpt-4o-search-preview-2025-03-

In [20]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4.1-mini")

tokens = encoding.encode("Hi my name is Ed and I like banoffee pie")

In [23]:
tokens = encoding.encode("Wawa, dzina langa ndine Dunstan Matekenya", disallowed_special=())

for m in tokens:
    tokenn_text = encoding.decode([m])
    print(f"{m}: {tokenn_text}")

54: W
7036: awa
11: ,
17303:  dz
1522: ina
6212:  lang
64: a
5914:  nd
514: ine
41163:  Dun
23648: stan
51888:  Mate
2144: ken
2090: ya


In [24]:
tokens = encoding.encode("Hi, my name is Dunstan Matekenya", disallowed_special=())

for m in tokens:
    tokenn_text = encoding.decode([m])
    print(f"{m}: {tokenn_text}")

12194: Hi
11: ,
922:  my
1308:  name
382:  is
41163:  Dun
23648: stan
51888:  Mate
2144: ken
2090: ya


In [7]:
tokens = encoding.encode("Mercy ndimakukonda kwambiri wamva", disallowed_special=())

In [9]:
len(tokens)

9

In [ ]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"{token_id} = {token_text}")

In [ ]:
encoding.decode([326])

# And another topic!

### The Illusion of "memory"

Many of you will know this already. But for those that don't -- this might be an "AHA" moment!

In [10]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### You should be very comfortable with what the next cell is doing!

_I'm creating a new instance of the OpenAI Python Client library, a lightweight wrapper around making HTTP calls to an endpoint for calling the GPT LLM, or other LLM providers_

In [11]:
from openai import OpenAI

openai = OpenAI()

### A message to OpenAI is a list of dicts

In [14]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Dunstan!"}
    ]

In [15]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Hello, Dunstan! How can I assist you today?'

### OK let's now ask a follow-up question

In [16]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
    ]

In [17]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'I’m sorry, but I don’t know your name. Could you please tell me?'

### Wait, wha??

We just told you!

What's going on??

Here's the thing: every call to an LLM is completely STATELESS. It's a totally new call, every single time. As AI engineers, it's OUR JOB to devise techniques to give the impression that the LLM has a "memory".

In [18]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Dunstan!"},
    {"role": "assistant", "content": "Hi Dunstan! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [19]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Your name is Dunstan. How can I help you today?'

## To recap

With apologies if this is obvious to you - but it's still good to reinforce:

1. Every call to an LLM is stateless
2. We pass in the entire conversation so far in the input prompt, every time
3. This gives the illusion that the LLM has memory - it apparently keeps the context of the conversation
4. But this is a trick; it's a by-product of providing the entire conversation, every time
5. An LLM just predicts the most likely next tokens in the sequence; if that sequence contains "My name is Ed" and later "What's my name?" then it will predict.. Ed!

The ChatGPT product uses exactly this trick - every time you send a message, it's the entire conversation that gets passed in.

"Does that mean we have to pay extra each time for all the conversation so far"

For sure it does. And that's what we WANT. We want the LLM to predict the next tokens in the sequence, looking back on the entire conversation. We want that compute to happen, so we need to pay the electricity bill for it!

